# 因子回测

## 导入模块

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import feather

## 读入日线数据

### 读入日线数据

In [2]:
price_1d = feather.read_dataframe('../data/StockPriceK1d_20240630.feather')

### 计算调仓日

In [3]:
trade_date = price_1d['date'].sort_values().unique()
mes = pd.date_range(start='2019-08-01', end=trade_date[-1], freq='1ME')
adj_date = np.array([], dtype=np.datetime64)
for me in mes:
    trade_date_before = trade_date[trade_date <= me]
    ad = trade_date_before[-1]
    adj_date = np.append(adj_date, ad)

### 剔除股票

In [4]:
def IPO_time(price_1d, timedelta='180D'):
    start_date = price_1d['date'].min()
    idx_ipo = price_1d['date'] >= start_date + pd.Timedelta(timedelta)
    return price_1d[idx_ipo]

price_1d = (
    price_1d
        .groupby('issue')[['issue', 'date', 'close', 'ret', 'is_limit_buy', 'is_limit_sell']]
        .apply(IPO_time)
        .reset_index(drop=True)
)
price_1d = price_1d[(~price_1d['is_limit_buy'].astype(bool)) & (~price_1d['is_limit_sell'].astype(bool))]
price_1d = price_1d[(price_1d['date'] >= '2019-08-01') & (price_1d['date'] <= '2024-11-30')]
price_1d = price_1d[['issue', 'date', 'ret']]

## IC 测试

### 读入测试数据

## 分组测试

## 点度中心性回测